# Differential Privacy for Vision Tasks

## Settings und Imports

In [1]:
# suppress warnings
import warnings

import opacus

warnings.filterwarnings('ignore')

#autoreload other packages when code changed
%load_ext autoreload
%autoreload 2

In [2]:
import torch

torch.manual_seed(20)  #Reproduzierbarkeit
from torch import nn
from torch.utils.data import DataLoader
import torchvision

from opacus import PrivacyEngine
from opacus.validators import ModuleValidator

from tqdm.notebook import tqdm
import gc

In [3]:
#Own Code
from privacyflow.configs import path_configs
from privacyflow.datasets import faces_dataset
from privacyflow.models import face_models

In [4]:
#Check if GPU is available
if torch.cuda.is_available():
    print("GPU will be used")
    device = torch.device('cuda')
else:
    print("No GPU available")
    device = torch.device('cpu')

GPU will be used


## Data Prep

In [5]:
label_columns = 'all'  #40 attributes

data_augmentation_train = torchvision.transforms.Compose([
    #torchvision.transforms.Resize((224,224)),
    torchvision.transforms.AutoAugment(),
    torchvision.transforms.ToTensor(),
])

data_augmentation_test = torchvision.transforms.Compose([
    #torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ToTensor()
])

data_augmentation_train_with_resize = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.AutoAugment(),
    torchvision.transforms.ToTensor(),
])

data_augmentation_test_with_resize = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor()
])


train_dataset_cnn = faces_dataset.FacesDataset(label_cols=label_columns, mode="train",transform=data_augmentation_train)
val_dataset_cnn = faces_dataset.FacesDataset(label_cols=label_columns, mode="val", transform=data_augmentation_test)
test_dataset_cnn = faces_dataset.FacesDataset(label_cols=label_columns, mode="test", transform=data_augmentation_test)

train_dataset_vit = faces_dataset.FacesDataset(label_cols=label_columns, mode="train",transform=data_augmentation_train_with_resize)
val_dataset_vit = faces_dataset.FacesDataset(label_cols=label_columns, mode="val",transform=data_augmentation_test_with_resize)
test_dataset_vit = faces_dataset.FacesDataset(label_cols=label_columns, mode="test",transform=data_augmentation_test_with_resize)

## Models - No DP 

In [6]:
def load_saved_model(model_type:str = "cnn", alt_path:str= None):
    if model_type == "cnn":
        model = face_models.get_FaceModelResNet(40)
        path = path_configs.FACE_BASE_MODEL if not alt_path else alt_path
        model.load_state_dict(torch.load(path))
        model = model.to(device)
        return model

    elif model_type == "transformer":
        model = face_models.get_FaceVisionTransformer(40)
        path = path_configs.FACE_VIT_MODEL if not alt_path else alt_path
        model.load_state_dict(torch.load(path))
        model = model.to(device)
        return model

    elif model_type == "dense":
        model = face_models.get_FaceModelDenseNet(40)
        path = path_configs.FACE_DENSE_MODEL if not alt_path else alt_path
        model.load_state_dict(torch.load(path))
        model = model.to(device)
        return model


In [7]:
def train_model(model:nn.Module,
                criterion:nn.Module,
                optimizer:torch.optim.Optimizer,
                epochs:int,
                train_ds:torch.utils.data.Dataset,
                val_ds:torch.utils.data.Dataset,
                batch_size:int =32,
                num_workers:int=0,
                amount_labels:int=40,
                val:bool=True):
    torch.cuda.empty_cache()
    train_dl = DataLoader(train_ds, batch_size=batch_size,shuffle=True,num_workers=num_workers)
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for model_inputs, labels in tqdm(train_dl, leave=False):
            model_inputs = model_inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            model_outputs = model(model_inputs)
            loss = criterion(model_outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch: {epoch + 1:2}",
              f"Train Loss: {epoch_loss / len(train_dl):.5f}")

        if val:
            torch.cuda.empty_cache()
            gc.collect() #free cuda memory from train_dataloader
            val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False,num_workers=num_workers)
            val_loss = 0.0
            num_corrects = 0
            model.eval()
            for model_inputs, labels in val_dl:
                model_inputs = model_inputs.to(device)
                labels = labels.to(device)
                model_outputs = model(model_inputs)
                loss = criterion(model_outputs, labels)
                val_loss += loss.item()
                num_corrects += int((model_outputs.round() == labels).sum())
            print(f"Val Loss: {val_loss / len(val_dl):.5f}",
                  f"Val Accuracy (all attributes): {num_corrects / (len(val_ds) * amount_labels)}")


def test_model(model:nn.Module,
               test_ds:torch.utils.data.Dataset,
               batch_size:int,
               num_workers:int =0,
               amount_labels=40):
    test_dl = DataLoader(test_ds, batch_size=batch_size,num_workers=num_workers,shuffle=False)
    model.eval()
    num_corrects = 0
    for model_inputs, labels in tqdm(test_dl):
        model_inputs = model_inputs.to(device)
        labels = labels.to(device)
        model_outputs = model(model_inputs)
        num_corrects += int((model_outputs.round() == labels).sum())
    print(f"Test Accuracy (all attributes): {num_corrects / (len(test_ds) * amount_labels)}")

In [ ]:
#ResNet not pretrained
model_cnn_base_np = face_models.get_FaceModelResNet(40,pretrained=False).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_cnn_base_np.parameters(), lr=0.01)

train_model(model=model_cnn_base_np,
            criterion=criterion,
            optimizer=optimizer,
            epochs=12,
            train_ds=train_dataset_cnn,
            val_ds=val_dataset_cnn,
            batch_size=128,
            num_workers=8,
            amount_labels=40,
            val=True)

# test_model(model=model_cnn_base_np, test_ds=test_dataset_cnn, batch_size=128, num_workers=8)
torch.save(model_cnn_base_np.state_dict(), path_configs.FACE_BASE_MODEL)
model_cnn_base_np.to(torch.device("cpu"))

In [ ]:
#ResNet
model_cnn_base = face_models.get_FaceModelResNet(40).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_cnn_base.parameters(), lr=0.01)

train_model(model=model_cnn_base,
            criterion=criterion,
            optimizer=optimizer,
            epochs=4,
            train_ds=train_dataset_cnn,
            val_ds=val_dataset_cnn,
            batch_size=128,
            num_workers=8,
            amount_labels=40)

test_model(model=model_cnn_base, test_ds=test_dataset_cnn, batch_size=128, num_workers=8)
torch.save(model_cnn_base.state_dict(), path_configs.FACE_BASE_MODEL)

In [ ]:
#Vision Transformer
model_vit_base = face_models.get_FaceVisionTransformer(40).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_vit_base.heads.parameters(), lr=0.01)

train_model(model=model_vit_base,
            criterion=criterion,
            optimizer=optimizer,
            epochs=3,
            train_ds=train_dataset_vit,
            val_ds=train_dataset_vit,
            batch_size=64,
            num_workers=8,
            amount_labels=40)

torch.save(model_vit_base.state_dict(), path_configs.FACE_VIT_MODEL)
test_model(model=model_vit_base, test_ds=test_dataset_vit, batch_size=64,num_workers=4)

In [ ]:
#Vision Transformer not pretrained
model_vit_base_np = face_models.get_FaceVisionTransformer(40,pretrained=False).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_vit_base_np.parameters(), lr=0.01)

train_model(model=model_vit_base_np,
            criterion=criterion,
            optimizer=optimizer,
            epochs=15,
            train_ds=train_dataset_vit,
            val_ds=train_dataset_vit,
            batch_size=128,
            num_workers=8,
            amount_labels=40,
            val=False)


torch.save(model_vit_base_np.state_dict(), path_configs.FACE_VIT_NP_MODEL)
test_model(model=model_vit_base_np, test_ds=test_dataset_vit, batch_size=64,num_workers=4)

## Model + DP-SGD

In [8]:
def train_model_dpsgd(model:nn.Module,
                criterion:nn.Module,
                optimizer:torch.optim.Optimizer,
                train_dl:torch.utils.data.DataLoader,
                privacy_engine:opacus.PrivacyEngine,
                val_dl:torch.utils.data.DataLoader = None,
                len_val_ds:int = 1,
                epochs:int = 5,
                amount_labels:int=40,
                max_epsilon:int= 10,
                val:bool=True):
    epsilon_reached = False
    for epoch in range(epochs):
        if epsilon_reached:
            break
        print(f"Start Training Epoch: {epoch + 1:2}")
        model.train()
        epoch_loss = 0.0
        for model_inputs, labels in tqdm(train_dl):
            try:
                model_inputs = model_inputs.to(device)
                labels = labels.to(device)
                #Forward + Backprop
                optimizer.zero_grad()
                model_outputs = model(model_inputs)
                loss = criterion(model_outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                #Check if epsilon exceeds threshold after each batch
                if max_epsilon < privacy_engine.accountant.get_epsilon(delta=1e-6):
                    print(f"ε Value {max_epsilon:2} reached in Epoch {epoch:2}")
                    epsilon_reached = True
                    break
            except RuntimeError:
                continue

        print(f"Finished Training Epoch: {epoch + 1:2}",
              f"Train Loss: {epoch_loss / len(train_dl):.5f}",
              f"ε:{privacy_engine.accountant.get_epsilon(delta=1e-6):.5f}")

        if val:
            torch.cuda.empty_cache()
            gc.collect() #free cuda memory from train_dataloader
            val_loss = 0.0
            num_corrects = 0
            model.eval()
            for model_inputs, labels in val_dl:
                try:
                    model_inputs = model_inputs.to(device)
                    labels = labels.to(device)
                    model_outputs = model(model_inputs)
                    loss = criterion(model_outputs, labels)
                    val_loss += loss.item()
                    num_corrects += int((model_outputs.round() == labels).sum())
                except RuntimeError:
                    continue
            print(f"Val Loss: {val_loss / len(val_dl):.5f}",
                  f"Val Accuracy (all attributes): {num_corrects / (len_val_ds * amount_labels)}")
        print("-------------------------------------------------------------")

In [10]:
#DPSGD consumes more memory, thus we decrease the batch size
train_dl_dpsgd = DataLoader(
    dataset=train_dataset_cnn,
    batch_size=64
)

In [11]:
for num_epochs in [1,3,5,10]:
    torch.cuda.empty_cache()
    gc.collect() #free cuda memory from train_dataloader
    model_cnn_dpsgd = face_models.get_FaceModelResNet(40,pretrained=False).to(device)
    model_cnn_dpsgd = ModuleValidator.fix(model_cnn_dpsgd)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model_cnn_dpsgd.parameters(), lr=0.01)
    privacy_engine= PrivacyEngine()
    model_cnn_dpsgd, optimizer, train_dl = privacy_engine.make_private_with_epsilon(
        module=model_cnn_dpsgd,
        optimizer=optimizer,
        data_loader=train_dl_dpsgd,
        epochs=num_epochs,
        target_epsilon=5,
        target_delta=1e-6,
        max_grad_norm=1.0 #Gradienten größer als dieser Wert werden geclippt
    )
    print(f"Num Epochs = {num_epochs} ,Noise Mult={optimizer.noise_multiplier}")
    torch.cuda.empty_cache()
    gc.collect() #free cuda memory from train_dataloader
    train_model_dpsgd(model=model_cnn_dpsgd,
                      criterion=criterion,
                      optimizer=optimizer,
                      train_dl=train_dl,
                      privacy_engine=privacy_engine,
                      max_epsilon=5,
                      epochs=num_epochs,
                      val=False)
    torch.save(model_cnn_dpsgd.state_dict(), f"{path_configs.MODELS_TRAINED_BASE_PATH}/cnn_epsilon5_epochs{num_epochs}_clipp1.pl" )


Num Epochs = 1 ,Noise Mult=0.44464111328125
Start Training Epoch:  1


  0%|          | 0/2544 [00:00<?, ?it/s]

Finished Training Epoch:  1 Train Loss: 0.44759 ε:4.99686
-------------------------------------------------------------
Num Epochs = 3 ,Noise Mult=0.463714599609375
Start Training Epoch:  1


  0%|          | 0/2544 [00:00<?, ?it/s]

Finished Training Epoch:  1 Train Loss: 0.44773 ε:4.32225
-------------------------------------------------------------
Start Training Epoch:  2


...

Finished Training Epoch:  8 Train Loss: 0.46067 ε:4.78454
-------------------------------------------------------------
Start Training Epoch:  9


  0%|          | 0/2544 [00:00<?, ?it/s]

Finished Training Epoch:  9 Train Loss: 0.46169 ε:4.88999
-------------------------------------------------------------
Start Training Epoch: 10


  0%|          | 0/2544 [00:00<?, ?it/s]

Finished Training Epoch: 10 Train Loss: 0.46185 ε:4.99018
-------------------------------------------------------------


In [12]:
for num_epochs in [1,3,5,10]:
    torch.cuda.empty_cache()
    gc.collect() #free cuda memory from train_dataloader
    model_cnn_dpsgd = face_models.get_FaceModelResNet(40,pretrained=False).to(device)
    model_cnn_dpsgd = ModuleValidator.fix(model_cnn_dpsgd)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model_cnn_dpsgd.parameters(), lr=0.01)
    privacy_engine= PrivacyEngine()
    model_cnn_dpsgd, optimizer, train_dl = privacy_engine.make_private_with_epsilon(
        module=model_cnn_dpsgd,
        optimizer=optimizer,
        data_loader=train_dl_dpsgd,
        epochs=num_epochs,
        target_epsilon=10,
        target_delta=1e-6,
        max_grad_norm=1.0 #Gradienten größer als dieser Wert werden geclippt
    )
    print(f"Num Epochs = {num_epochs} ,Noise Mult={optimizer.noise_multiplier}")
    torch.cuda.empty_cache()
    gc.collect() #free cuda memory from train_dataloader
    train_model_dpsgd(model=model_cnn_dpsgd,
                      criterion=criterion,
                      optimizer=optimizer,
                      train_dl=train_dl,
                      privacy_engine=privacy_engine,
                      max_epsilon=10,
                      epochs=num_epochs,
                      val=False)
    torch.save(model_cnn_dpsgd.state_dict(), f"{path_configs.MODELS_TRAINED_BASE_PATH}/cnn_epsilon10_epochs{num_epochs}_clipp1.pl" )

Num Epochs = 1 ,Noise Mult=0.360107421875
Start Training Epoch:  1


  0%|          | 0/2544 [00:00<?, ?it/s]

Finished Training Epoch:  1 Train Loss: 0.44450 ε:9.99461
-------------------------------------------------------------
Num Epochs = 3 ,Noise Mult=0.38299560546875
Start Training Epoch:  1


  0%|          | 0/2544 [00:00<?, ?it/s]

Finished Training Epoch:  1 Train Loss: 0.44475 ε:8.13555
-------------------------------------------------------------
Start Training Epoch:  2


...

Finished Training Epoch:  4 Train Loss: 0.44459 ε:9.49626
-------------------------------------------------------------
Start Training Epoch:  5


  0%|          | 0/2544 [00:00<?, ?it/s]

Finished Training Epoch:  5 Train Loss: 0.44377 ε:9.99608
-------------------------------------------------------------
Num Epochs = 10 ,Noise Mult=0.41229248046875
Start Training Epoch:  1


  0%|          | 0/2544 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [47]:
for num_epochs in [1,3,5,10]:
    torch.cuda.empty_cache()
    gc.collect() #free cuda memory from train_dataloader
    model_cnn_dpsgd = face_models.get_FaceModelResNet(40,pretrained=False).to(device)
    model_cnn_dpsgd = ModuleValidator.fix(model_cnn_dpsgd)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model_cnn_dpsgd.parameters(), lr=0.01)
    privacy_engine= PrivacyEngine()
    model_cnn_dpsgd, optimizer, train_dl = privacy_engine.make_private_with_epsilon(
        module=model_cnn_dpsgd,
        optimizer=optimizer,
        data_loader=train_dl_dpsgd,
        epochs=num_epochs,
        target_epsilon=1,
        target_delta=1e-6,
        max_grad_norm=0.5 #Gradienten größer als dieser Wert werden geclippt
    )
    print(f"Num Epochs = {num_epochs} ,Noise Mult={optimizer.noise_multiplier}")
    torch.cuda.empty_cache()
    gc.collect() #free cuda memory from train_dataloader
    train_model_dpsgd(model=model_cnn_dpsgd,
                      criterion=criterion,
                      optimizer=optimizer,
                      train_dl=train_dl,
                      privacy_engine=privacy_engine,
                      max_epsilon=1,
                      epochs=num_epochs,
                      val=False)
    torch.save(model_cnn_dpsgd.state_dict(), f"{path_configs.MODELS_TRAINED_BASE_PATH}/cnn_epochs{num_epochs}_clipp0-5.pl" )

Num Epochs = 1 ,Noise Mult=0.631103515625
Start Training Epoch:  1


  0%|          | 0/2544 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [39]:
model_cnn_dpsgd = face_models.get_FaceModelResNet(40,pretrained=False).to(device)
model_cnn_dpsgd = ModuleValidator.fix(model_cnn_dpsgd)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_cnn_dpsgd.parameters(), lr=0.01)

In [30]:
privacy_engine= PrivacyEngine()
model_cnn_dpsgd, optimizer, train_dl = privacy_engine.make_private(
    module=model_cnn_dpsgd,
    optimizer=optimizer,
    data_loader=train_dl_dpsgd,
    noise_multiplier=1.0, #Wie viel Rauschen wird hinzugefügt - Höher = weniger Rauschen
    max_grad_norm=1.0 #Gradienten größer als dieser Wert werden geclippt
)

In [31]:
train_model_dpsgd(model=model_cnn_dpsgd,
                  criterion=criterion,
                  optimizer=optimizer,
                  train_dl=train_dl,
                  privacy_engine=privacy_engine,
                  max_epsilon=1,
                  epochs=10,
                  val=False)

Start Training Epoch:  1


  0%|          | 0/1272 [00:00<?, ?it/s]

Finished Training Epoch:  1 Train Loss: 0.44029 ε:0.16868
-------------------------------------------------------------
Start Training Epoch:  2


  0%|          | 0/1272 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model_cnn_dpsgd.train()
for epoch in range(3):
    for dp_inputs,labels in tqdm(train_dl):
        try:
            dp_inputs = dp_inputs.to(device)
            labels = labels.to(device)

            model_cnn_dpsgd.zero_grad()
            optimizer.zero_grad()
            outputs = model_cnn_dpsgd(dp_inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        except RuntimeError as error:
            continue
        except:
            break
    print(f"Finished Training Epoch {epoch+1}")
    print(f"ε Value {privacy_engine.accountant.get_epsilon(delta=1e-6):.5f}")


In [ ]:
data_augmentation_train = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

train_dataset_cnn = faces_dataset.FacesDataset(label_cols=label_columns, mode="train",
                                               transform=data_augmentation_train)

train_dataset_cnn = faces_dataset.FacesDataset(label_cols=label_columns, mode="val",
                                               transform=data_augmentation_train)
train_dataloader_cnn = DataLoader(
    dataset=train_dataset_cnn,
    batch_size=4,
    shuffle=True
)
val_dataloader_cnn = DataLoader(
    dataset=val_dataset_cnn,
    batch_size=4,
    shuffle=False
)
test_dataloader_cnn = DataLoader(
    dataset=test_dataset_cnn,
    batch_size=4,
    shuffle=False
)

In [ ]:
model_cnn_dpsgd = face_models.get_FaceModelResNet(40).to(device)
model_cnn_dpsgd = ModuleValidator.fix(model_cnn_dpsgd)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_cnn_dpsgd.fc.parameters(), lr=0.01)


#test_model(model=model_cnn_base, test_dl=test_dataloader_cnn, len_test_ds=len(test_dataset_cnn))

#torch.save(model_cnn_base.state_dict(), path_configs.FACE_BASE_MODEL)